# Toronto Neighborhood Clustering


In [14]:
#importing modules
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [15]:
toronto_neighbor_page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [16]:
soup = BeautifulSoup(toronto_neighbor_page, 'html')
#print(soup.prettify())

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /opt/conda/envs/DSX-Python35/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [18]:
mytable = soup.find('table')

In [20]:
cols = mytable.find_all('td')

In [23]:
# creating a list of postal codes, borough and neighborhood by scraping using 
postal_code = []
borough = []
neighborhood = []
for info in cols:
    if info.find('span').find('i'):
        continue
    else:
        postal_code.append(info.find('b').text)
        text = info.find('span').text
        idx = text.find('(')
        if idx != -1:
            borough.append(text[:idx])
            neighborhood.append(text[idx+1: ])
        else:
            borough.append('')
            neighborhood.append('')
#print(postal_code, borough, neighborhood)
neighborhoods = []
for neighbor in neighborhood:
    neighborhoods.append(neighbor.replace('/', ',').replace(')', ' ').replace('(', ',').rstrip())
#print("nrighbor = ", neighborhoods)
#print(len(postal_code), len(borough), len(neighborhoods))


In [24]:
# PostalCode, Borough, and Neighborhood
df = pd.DataFrame({'PostalCode':postal_code, 'Borough':borough, 'Neighborhood': neighborhoods})
df

,Borough,Neighborhood,PostalCode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,"Regent Park , Harbourfront",M5A
3,North York,"Lawrence Manor , Lawrence Heights",M6A
4,,,M7A
5,Etobicoke,Islington Avenue,M9A
6,Scarborough,"Malvern , Rouge",M1B
7,North York,Don Mills North,M3B
8,East York,"Parkview Hill , Woodbine Gardens",M4B
9,Downtown Toronto,"Garden District, Ryerson",M5B


In [34]:
#getting values of dataframe columns to pass in the google's geocoding api
postal_codes = df['PostalCode'].get_values()
neighborhood = df['Neighborhood'].get_values()
#print(postal_codes, len(postal_codes))
#print(neighborhood, len(neighborhood))


In [35]:
# The code was removed by Watson Studio for sharing.

In [36]:
# Getting the latitude and longitude using google geocoding api 
locations = []
# construct URL to make API call
for idx in range(len(postal_codes)):
    #print(postal_codes[idx] + "," + neighborhood[idx])
    url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(API_key, postal_codes[idx] + "," + neighborhood[idx])
    response = requests.get(url).json() # get response
    #print(response)
    if len(response['results']) > 0:
        geographical_data = response['results'][0]['geometry']['location'] # get geographical coordinates
        latitude = geographical_data['lat']
        longitude = geographical_data['lng']
        #print(latitude, longitude)
        locations.append((latitude, longitude))
    else:
        latitude = ''
        longitude = ''
        locations.append((latitude, longitude))
        continue
#print(locations)

[(43.7532586, -79.3296565), (43.72588229999999, -79.3155716), (43.638834, -79.3818785), (43.72277400000001, -79.45092799999999), ('', ''), (43.6619767, -79.5309608), (43.8066863, -79.1943534), (43.7459058, -79.352188), (43.7061385, -79.3254151), (43.6576585, -79.3788017), (43.7087787, -79.4407294), (43.7213234, -79.5845715), (43.78453510000001, -79.1604971), (43.7211861, -79.3389695), (43.693328, -79.316064), (43.6708666, -79.37330639999999), (43.6937813, -79.4281914), (43.643238, -79.580168), (43.7929401, -79.19598119999999), (43.6763574, -79.2930312), (43.6479848, -79.37522530000001), (43.6869159, -79.45632499999999), (43.767528, -79.21772100000001), (43.7090604, -79.3634517), (43.6561311, -79.3842646), (43.6727456, -79.4219686), (43.773136, -79.23947609999999), (43.8037622, -79.3634517), (43.7481449, -79.45464919999999), (43.7053689, -79.3493719), (43.6496814, -79.3837757), (43.8446696, -79.48354859999999), (43.7447342, -79.23947609999999), (43.77178199999999, -79.352086), (43.77345

In [61]:
lats = []
longs = []
for location in locations:
    lat = location[0]
    long = location[1]
    lats.append(lat)
    longs.append(long)
df['Latitude'] = lats
df['Longitude'] = longs
df.loc[89, ["Latitude"]] = 43.7394164
df.loc[89, "Longitude"] = -79.58843689999999
df.loc[76, ["Latitude"]] = 43.63827828029151
df.loc[76, "Longitude"] = -79.61459401970849
print(df[df["Latitude"] == '' & df["Longitude"] == ''])
print(type(df.loc[4, "Latitude"]))

TypeError: cannot compare a dtyped [object] array with a scalar of type [bool]

In [50]:
url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(API_key, "M7R, Enclave of L4W, MississaugaCanada Post Gateway Processing Centre")
response = requests.get(url).json() # get response
print(response)

{'status': 'OK', 'results': [{'address_components': [{'short_name': '4567', 'types': ['street_number'], 'long_name': '4567'}, {'short_name': 'Dixie Rd', 'types': ['route'], 'long_name': 'Dixie Road'}, {'short_name': 'Mississauga', 'types': ['locality', 'political'], 'long_name': 'Mississauga'}, {'short_name': 'Peel Regional Municipality', 'types': ['administrative_area_level_2', 'political'], 'long_name': 'Peel Regional Municipality'}, {'short_name': 'ON', 'types': ['administrative_area_level_1', 'political'], 'long_name': 'Ontario'}, {'short_name': 'CA', 'types': ['country', 'political'], 'long_name': 'Canada'}, {'short_name': 'L4W 1S2', 'types': ['postal_code'], 'long_name': 'L4W 1S2'}], 'formatted_address': '4567 Dixie Rd, Mississauga, ON L4W 1S2, Canada', 'types': ['establishment', 'finance', 'point_of_interest', 'post_office'], 'partial_match': True, 'plus_code': {'compound_code': 'J9PM+QJ Mississauga, Ontario, Canada', 'global_code': '87M2J9PM+QJ'}, 'geometry': {'location': {'lat

In [ ]:
df['']